#### [ForbidExtent()](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.forbid_extent) vs [Stepwise Functions](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.function.py.html#docplex.cp.function.CpoStepFunction)

In Constraint Programming (`docplex.cp`), it is crucial to distinguish between **defining a state** over time and **enforcing a rule** based on that state.

1. The Concepts at a Glance

A. Stepwise Function (`CpoStepFunction`)
* **Type:** Data Structure (Passive).
* **Definition:** A function $f(t)$ that defines values over time.
* **Role:** It acts as a **Map**. It describes the environment (e.g., "The machine is broken from 10:00 to 12:00").
* **Effect:** On its own, it does **nothing** to the schedule. It just holds information.

B. Forbid Extent (`forbid_extent`)
* **Type:** Constraint (Active).
* **Definition:** A rule applied to the model.
* **Role:** It acts as the **Enforcer**. It looks at a Stepwise Function and applies a restriction.
* **Effect:** It forces the solver to ensure a task **never overlaps** with any time segment where the Stepwise Function has a value of `0`.


1. The "Wet Cement" Analogy

To understand the relationship, imagine a sidewalk construction project:

| Component | Analogy | Explanation |
| :--- | :--- | :--- |
| **Stepwise Function** | **The Warning Sign** | It marks specific times/areas as "Wet Cement" (Value 0) or "Dry Concrete" (Value 100). It identifies the danger zones but doesn't physically stop you. |
| **Forbid Extent** | **The Security Guard** | The guard looks at the Warning Sign and physically prevents people (Tasks) from stepping on the wet cement. |

In [ ]:
from docplex.cp.model import CpoModel, CpoStepFunction, interval_var, forbid_extent

mdl = CpoModel()

# --- PART 1: The Map (Stepwise Function) ---
# We define that a resource is unavailable (0) between time 10 and 20.
# This is just data. The solver doesn't care yet.
availability = CpoStepFunction()
availability.set_value(0, 100)   # Available
availability.set_value(10, 0)    # UNAVAILABLE (The "Hole")
availability.set_value(20, 100)  # Available again

# --- PART 2: The Actor (Interval Variable) ---
# A task that takes 5 units of time.
task = interval_var(length=5, name="MyTask")

# --- PART 3: The Rule (Forbid Extent) ---
# This effectively says: "Make sure 'task' does not fall into the 'Hole'."
# Without this line, the solver could schedule the task at t=12.
mdl.add(forbid_extent(task, availability))

mdl.solve()

[ICAPS 2017: Video Tutorial – Philippe Laborie: Introduction to CP Optimizer for Scheduling](https://www.youtube.com/watch?v=-VY7QTnSAio), [Slides from the video](https://icaps17.icaps-conference.org/tutorials/T3-Introduction-to-CP-Optimizer-for-Scheduling.pdf)

slide 42: interval variable size, length adn intensity funciton

what for? 
- modeling cases where the intensity of work is not the same duting the whole interval and the interval requires some quantity of work to be done before completion

- activities that are suspended during some time periods (e.g. week-end, vacation)

- https://www.ibm.com/docs/en/icos/22.1.2?topic=scheduling-interval-variables

How to model that?

- Stretch tasks across closed periods (simple “calendar” via intensity)
- When to use: Tasks are not preemptable and simply “pause” during closed time (e.g., a 6-hour job that starts at 10:00 spans over a 12:00–13:00 lunch and finishes at 17:00).

Make each task’s processing requirement the size.

Give tasks an intensity step function that is 100 when working and 0 when blocked.
CP Optimizer will automatically lengthen the interval to cover the size only during periods with intensity > 0.

In [ ]:
mdl = CpoModel()

# Example data
jobs = ["A", "B"]
durations = {"A": 6, "B": 4}  # required work time in "work units" (e.g. hours)

# Define a step function for working hours
# open 8:00–12:00, 13:00–17:00 (minutes since midnight)
work_cal = CpoStepFunction()
work_cal.set_value(0, 8*60, 0)      # before 8:00 off
work_cal.set_value(8*60, 12*60, 100)
work_cal.set_value(12*60, 13*60, 0)
work_cal.set_value(13*60, 17*60, 100)
work_cal.set_value(17*60, 24*60, 0)

# Create interval variables with that intensity
tasks = {j: mdl.interval_var(size=durations[j], intensity=work_cal, name=j)
         for j in jobs}

# Single machine
mdl.add(mdl.no_overlap(list(tasks.values())))

# Example objective (minimize makespan)
mdl.add(mdl.minimize(mdl.max([mdl.end_of(t) for t in tasks.values()])))


Forbid any overlap with closed periods (forbidExtent)

When to use: You want to prohibit a task from covering any closed time at all (it must sit entirely inside open windows).

 - Build a “closed” step function that is 1 during blocked time and 0 otherwise.

- Use forbidExtent so [start, end) of each interval never intersects a blocked segment.

In [ ]:
mdl = CpoModel()

jobs = ["A", "B"]
durations = {"A": 6, "B": 4}

# Define a "closed" step function: 1 during blocked periods, 0 otherwise
closed = CpoStepFunction()
closed.set_value(0, 8*60, 1)
closed.set_value(8*60, 12*60, 0)
closed.set_value(12*60, 13*60, 1)
closed.set_value(13*60, 17*60, 0)
closed.set_value(17*60, 24*60, 1)

tasks = {j: mdl.interval_var(size=durations[j], name=j) for j in jobs}

# Forbid tasks from overlapping blocked times
for j in jobs:
    mdl.add(mdl.forbid_extent(tasks[j], closed))

mdl.add(mdl.no_overlap(list(tasks.values())))
mdl.add(mdl.minimize(mdl.max([mdl.end_of(t) for t in tasks.values()])))


Insert downtime as fixed intervals and share the resource (noOverlap with extra intervals)

When to use: You want downtime to consume the same resource just like a job (e.g., planned maintenance blocks the machine).

- Create fixed, present interval variables for each blocking window.

- Put both jobs and downtime intervals into the same noOverlap.

In [ ]:
mdl = CpoModel()

jobs = ["A", "B"]
durations = {"A": 6, "B": 4}

# Known downtime intervals (start, end)
downtimes = [(12*60, 13*60)]  # 12:00–13:00

tasks = {j: mdl.interval_var(size=durations[j], name=j) for j in jobs}

# Create fixed blocking intervals
blocks = []
for i, (s, e) in enumerate(downtimes):
    iv = mdl.interval_var(start=s, end=e, name=f"block_{i}")
    mdl.add(mdl.presence_of(iv) == 1)
    blocks.append(iv)

# Combine jobs + blocks in same machine constraint
mdl.add(mdl.no_overlap(list(tasks.values()) + blocks))

mdl.add(mdl.minimize(mdl.max([mdl.end_of(t) for t in tasks.values()])))

forbidStart(), forbidEnd(), forbidExtent()